In [1]:
import datajoint as dj
import numpy as np
import json
from subprocess import call
import yaml
import sys, os
import pandas as pd
import ray

sys.path.append('./lib')
from utilities import *
sys.path.append('../lib')
from utils import run

In [2]:
#credFiles= '/data/Github/VaultBrain/credFiles_aws.yaml'
credFiles= '/Users/kuiqian/Github/VaultBrain/credFiles.yaml'
dj.config['database.host'] = get_dj_creds(credFiles)['database.host']
dj.config['database.user'] = get_dj_creds(credFiles)['database.user']
dj.config['database.port'] = get_dj_creds(credFiles)['database.port']
dj.config['database.password'] = get_dj_creds(credFiles)['database.password']
dj.conn()

./lib/utilities.py:14: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  credFiles = yaml.load(open( credential_file_pointers,'r'))


Connecting kui@ucsd-demo-db.datajoint.io:3306


DataJoint connection (connected) kui@ucsd-demo-db.datajoint.io:3306

In [3]:
schema = dj.schema('kui_diffusionmap')
schema.spawn_missing_classes()

In [4]:
stack = 'MD594'
#yaml_file = 'shape_params-aws.yaml'
yaml_file = 'shape_params.yaml'
scripts_dir = os.environ['REPO_DIR']

In [10]:
fname = os.path.join('CSHL_data_processed', stack, 'Annotation.npy')
annotation = np.load(os.environ['ROOT_DIR']+fname, allow_pickle = True, encoding='latin1')
contours = pd.DataFrame(annotation)
contours = contours.rename(columns={0:"name", 1:"section", 2:"vertices"})
contours_grouped = contours.groupby('section')

In [11]:
sections = np.sort(contours['section'].unique())
sections

array([94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107,
       108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120,
       121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 132, 133, 134,
       135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147,
       148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160,
       161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173,
       174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186,
       188, 189, 190, 191, 192, 194, 195, 196, 197, 198, 199, 200, 201,
       202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214,
       215, 216, 217, 218, 219, 220, 221, 222, 223, 224, 225, 226, 227,
       228, 229, 230, 231, 232, 233, 234, 235, 236, 237, 239, 240, 241,
       242, 243, 244, 245, 246, 247, 248, 249, 250, 251, 252, 253, 254,
       255, 256, 257, 258, 259, 260, 262, 263, 264, 265, 266, 267, 268,
       269, 270, 271, 272, 273, 274, 275, 276, 277, 278, 279, 280

In [12]:
np.where(sections==256)

(array([158]),)

In [8]:
sections[140]

237

In [8]:
SectionV2.drop()

`kui_diffusionmap`.`section_v2` (45 tuples)
`kui_diffusionmap`.`__cell_mark` (45 tuples)
Proceed? [yes, No]: yes
Tables dropped.  Restart kernel.


In [9]:
@schema
class SectionV2(dj.Manual):
    definition = """
    section_id  : int   # section number
    """

In [10]:
valid_sections = []
for section in sections:
    if contours_grouped.get_group(section).iterrows():
        valid_sections.append(section)

In [11]:
len(valid_sections), sections.shape

(265, (265,))

In [12]:
for section in valid_sections:
    print("\nAdding "+str(section)+' to the database')
    SectionV2.insert1(dict(section_id=section)
                      ,skip_duplicates=True)


Adding 94 to the database

Adding 95 to the database

Adding 96 to the database

Adding 97 to the database

Adding 98 to the database

Adding 99 to the database

Adding 100 to the database

Adding 101 to the database

Adding 102 to the database

Adding 103 to the database

Adding 104 to the database

Adding 105 to the database

Adding 106 to the database

Adding 107 to the database

Adding 108 to the database

Adding 109 to the database

Adding 110 to the database

Adding 111 to the database

Adding 112 to the database

Adding 113 to the database

Adding 114 to the database

Adding 115 to the database

Adding 116 to the database

Adding 117 to the database

Adding 118 to the database

Adding 119 to the database

Adding 120 to the database

Adding 121 to the database

Adding 122 to the database

Adding 123 to the database

Adding 124 to the database

Adding 125 to the database

Adding 126 to the database

Adding 127 to the database

Adding 128 to the database

Adding 129 to the databas

In [6]:
SectionV2.fetch(as_dict=True)

[OrderedDict([('section_id', 94)]),
 OrderedDict([('section_id', 95)]),
 OrderedDict([('section_id', 96)]),
 OrderedDict([('section_id', 97)]),
 OrderedDict([('section_id', 98)]),
 OrderedDict([('section_id', 99)]),
 OrderedDict([('section_id', 100)]),
 OrderedDict([('section_id', 101)]),
 OrderedDict([('section_id', 102)]),
 OrderedDict([('section_id', 103)]),
 OrderedDict([('section_id', 104)]),
 OrderedDict([('section_id', 105)]),
 OrderedDict([('section_id', 106)]),
 OrderedDict([('section_id', 107)]),
 OrderedDict([('section_id', 108)]),
 OrderedDict([('section_id', 109)]),
 OrderedDict([('section_id', 110)]),
 OrderedDict([('section_id', 111)]),
 OrderedDict([('section_id', 112)]),
 OrderedDict([('section_id', 113)]),
 OrderedDict([('section_id', 114)]),
 OrderedDict([('section_id', 115)]),
 OrderedDict([('section_id', 116)]),
 OrderedDict([('section_id', 117)]),
 OrderedDict([('section_id', 118)]),
 OrderedDict([('section_id', 119)]),
 OrderedDict([('section_id', 120)]),
 Ordere

In [7]:
@schema
class SectionV3(dj.Manual):
    definition = """
    section_id  : int   # section number
    """

In [9]:
valid_sections = [256, 257, 258, 259, 260, 237]
for section in valid_sections:
    print("\nAdding "+str(section)+' to the database')
    SectionV3.insert1(dict(section_id=section)
                      ,skip_duplicates=True)


Adding 256 to the database

Adding 257 to the database

Adding 258 to the database

Adding 259 to the database

Adding 260 to the database

Adding 237 to the database


In [10]:
SectionV3.fetch(as_dict=True)

[OrderedDict([('section_id', 237)]),
 OrderedDict([('section_id', 256)]),
 OrderedDict([('section_id', 257)]),
 OrderedDict([('section_id', 258)]),
 OrderedDict([('section_id', 259)]),
 OrderedDict([('section_id', 260)])]

In [31]:
schema.jobs

table_name className of the table,key_hash key hash,"status if tuple is missing, the job is available",key structure containing the key,error_message error message returned if failed,error_stack error stack if failed,user database user,host system hostname,pid system process id,connection_id connection_id(),timestamp automatic timestamp


In [72]:
(schema.jobs & 'status="error"')

table_name className of the table,key_hash key hash,"status if tuple is missing, the job is available",key structure containing the key,error_message error message returned if failed,error_stack error stack if failed,user database user,host system hostname,pid system process id,connection_id connection_id(),timestamp automatic timestamp


In [73]:
(schema.jobs & 'status="reserved"' )

table_name className of the table,key_hash key hash,"status if tuple is missing, the job is available",key structure containing the key,error_message error message returned if failed,error_stack error stack if failed,user database user,host system hostname,pid system process id,connection_id connection_id(),timestamp automatic timestamp


In [10]:
(schema.jobs & 'host="Kuis-MacBook-Pro.local"' )

table_name className of the table,key_hash key hash,"status if tuple is missing, the job is available",key structure containing the key,error_message error message returned if failed,error_stack error stack if failed,user database user,host system hostname,pid system process id,connection_id connection_id(),timestamp automatic timestamp
__shift,a8baa56554f96369ab93e4f3bb068c22,reserved,=BLOB=,,=BLOB=,kui@76.176.26.71,Kuis-MacBook-Pro.local,16956,21533,2019-07-18 08:51:00


In [14]:
(schema.jobs & 'host="ip-172-31-31-242"' )

table_name className of the table,key_hash key hash,"status if tuple is missing, the job is available",key structure containing the key,error_message error message returned if failed,error_stack error stack if failed,user database user,host system hostname,pid system process id,connection_id connection_id(),timestamp automatic timestamp
__shift,006f52e9102a8d3be2fe5614f42ba989,error,=BLOB=,DataJointError: Connection was lost during a transaction.,=BLOB=,kui@172.31.31.242,ip-172-31-31-242,14518,39508,2019-09-06 21:12:32
__shift,9dcb88e0137649590b755372b040afad,reserved,=BLOB=,,=BLOB=,kui@172.31.31.242,ip-172-31-31-242,14518,39508,2019-09-06 21:12:32


In [59]:
(schema.jobs & 'status="error"' ).delete()

In [30]:
(schema.jobs & 'status="reserved"' ).delete()
(schema.jobs & 'status="error"' ).delete()

In [6]:
CellMark.drop()

`kui_diffusionmap`.`__cell_mark` (6 tuples)
Proceed? [yes, No]: yes
Tables dropped.  Restart kernel.


In [10]:
CellMark.fetch(as_dict=True)

[OrderedDict([('section_id', 237), ('structure_number', 1)]),
 OrderedDict([('section_id', 256), ('structure_number', 1)]),
 OrderedDict([('section_id', 257), ('structure_number', 1)]),
 OrderedDict([('section_id', 258), ('structure_number', 1)]),
 OrderedDict([('section_id', 259), ('structure_number', 1)]),
 OrderedDict([('section_id', 260), ('structure_number', 1)])]

In [5]:
Shift.fetch(as_dict=True)

[OrderedDict([('section_id', 94), ('size_of_file', 3)]),
 OrderedDict([('section_id', 95), ('size_of_file', 3)]),
 OrderedDict([('section_id', 96), ('size_of_file', 3)]),
 OrderedDict([('section_id', 97), ('size_of_file', 3)]),
 OrderedDict([('section_id', 98), ('size_of_file', 3)]),
 OrderedDict([('section_id', 99), ('size_of_file', 3)]),
 OrderedDict([('section_id', 100), ('size_of_file', 3)]),
 OrderedDict([('section_id', 101), ('size_of_file', 3)]),
 OrderedDict([('section_id', 102), ('size_of_file', 3)]),
 OrderedDict([('section_id', 103), ('size_of_file', 3)]),
 OrderedDict([('section_id', 104), ('size_of_file', 3)]),
 OrderedDict([('section_id', 105), ('size_of_file', 13)]),
 OrderedDict([('section_id', 106), ('size_of_file', 13)]),
 OrderedDict([('section_id', 107), ('size_of_file', 13)]),
 OrderedDict([('section_id', 108), ('size_of_file', 13)]),
 OrderedDict([('section_id', 109), ('size_of_file', 13)]),
 OrderedDict([('section_id', 110), ('size_of_file', 13)]),
 OrderedDict([

In [8]:
Shift.fetch(as_dict=True)

[OrderedDict([('section_id', 94), ('size_of_file', 3)]),
 OrderedDict([('section_id', 95), ('size_of_file', 3)]),
 OrderedDict([('section_id', 96), ('size_of_file', 3)]),
 OrderedDict([('section_id', 97), ('size_of_file', 3)]),
 OrderedDict([('section_id', 98), ('size_of_file', 3)]),
 OrderedDict([('section_id', 99), ('size_of_file', 3)]),
 OrderedDict([('section_id', 100), ('size_of_file', 3)]),
 OrderedDict([('section_id', 101), ('size_of_file', 3)]),
 OrderedDict([('section_id', 102), ('size_of_file', 3)]),
 OrderedDict([('section_id', 103), ('size_of_file', 3)]),
 OrderedDict([('section_id', 104), ('size_of_file', 3)]),
 OrderedDict([('section_id', 105), ('size_of_file', 13)]),
 OrderedDict([('section_id', 106), ('size_of_file', 13)]),
 OrderedDict([('section_id', 107), ('size_of_file', 13)]),
 OrderedDict([('section_id', 108), ('size_of_file', 13)]),
 OrderedDict([('section_id', 109), ('size_of_file', 13)]),
 OrderedDict([('section_id', 110), ('size_of_file', 13)]),
 OrderedDict([

In [6]:
Shift.drop()

`kui_diffusionmap`.`__shift` (179 tuples)
Proceed? [yes, No]: yes
Tables dropped.  Restart kernel.


In [27]:
! python Shift_datajoint.py 'local' 'MD594'

./lib/utilities.py:14: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  credFiles = yaml.load(open( credential_file_pointers,'r'))
Connecting kui@ucsd-demo-db.datajoint.io:3306
ALREADY DOWNLOADED FILE
ALREADY DOWNLOADED FILE
./lib/utilities.py:27: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  credFiles = yaml.load(open( credential_file_pointers,'r'))
populating for  364
run cmd= python /Users/kuiqian/Github/shapeology_code/scripts//Shape_shift_V2.py MD594 364
/Users/kuiqian/Github/venv/shapeology_venv/lib/python3.7/site-packages/sklearn/externals/joblib/externals/cloudpickle/cloudpickle.py:47: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  import imp
ALREADY DOWNLOADED FILE
ALRE

In [7]:
schema.size_on_disk

65536

In [88]:
schema.connection.close()

In [93]:
schema.drop('kui_diffusionmap')

In [12]:
schema.log

timestamp,version datajoint version,user user@host,host system hostname,event custom message
2019-07-18 06:50:51,0.11.2py,kui@76.176.26.71,Kuis-MacBook-Pro.local,DELETE FROM `kui_diffusionmap`.`~log`
